In [1]:
# Fun some setup for the project
# Silence annoying pytorch deprecated warnings
import warnings
warnings.filterwarnings("ignore")

from train import *
from create_graphs import *
%matplotlib inline

print ('CUDA Available:', torch.cuda.is_available())

CUDA Available: True


In [2]:
# Generate the model datasets (i.e. the dataset used to train
# the model - normal data)
args_enzy1, train_enzy1, val_enzy1, test_enzy1 = get_graph_data("enzymes_1", isModelDataset=True)
# Save the max_previous node to allow for model 
# compatability on future datasets
max_prev_node = args_enzy1.max_prev_node

Loading graph dataset: ENZYMES
Loaded
Average num nodes 36.505050505050505
total graph num: 99
max number node: 125
max/min number edge: 149; 26
max previous node: 25
